<div dir="rtl">
أكيد، هجمع لك كل التفاصيل خطوة بخطوة، بحيث تفهم الصورة الكاملة حول **OAuth2PasswordBearer** وازاي بيشتغل مع التوكنات في **FastAPI**. 🚀

---

## 🎯 **ما هو OAuth2PasswordBearer؟**
هو **Dependency** في FastAPI يُستخدم لاستخراج **الـ access token** من طلبات **HTTP Bearer Token Authentication**.
بمعنى تاني، لما أي مستخدم يحاول يدخل للـ API، لازم يبعت الـ **access token** في الهيدر، والـ `OAuth2PasswordBearer` بياخد التوكن من هناك.

---

## 🛠 **خطوات تنفيذ Authentication باستخدام OAuth2PasswordBearer**
### ✅ **1. تعريف `OAuth2PasswordBearer`**
أول حاجة بنحدد إننا هنستخدم نظام الـ Bearer Tokens عن طريق:
```python
from fastapi.security import OAuth2PasswordBearer

oauth2_bearer = OAuth2PasswordBearer(tokenUrl="auth/token")
```
📌 **ليه كتبنا `tokenUrl="auth/token"`؟**
لأننا بنقول لـ FastAPI إن المستخدم هيبعت الـ token اللي خدناه من **endpoint معين** وهو `/auth/token` (يعني المكان اللي بيحصل فيه تسجيل الدخول وإرجاع التوكن).

---

### ✅ **2. إنشاء Access Token**
لما المستخدم يسجل دخوله بنبعت له توكن. بنعملها باستخدام:
```python
from datetime import datetime, timedelta, timezone
from jose import jwt

SECRET_KEY = "your_secret_key"
ALGORITHM = "HS256"

def create_access_token(username: str, user_id: int, expires_delta: timedelta):
    payload = {
        "username": username,
        "user_id": user_id,
        "exp": datetime.now(timezone.utc) + expires_delta
    }
    return jwt.encode(payload, SECRET_KEY, algorithm=ALGORITHM)
```
📌 **إيه اللي بيحصل هنا؟**
- بنعمل **JWT Token** يحتوي على:
  - `username`: اسم المستخدم
  - `user_id`: الـ ID بتاعه
  - `exp`: وقت انتهاء الصلاحية
- بعد كده بنعمل **تشفير (Encoding)** للتوكن باستخدام **مفتاح سري (`SECRET_KEY`)** و **خوارزمية (`HS256`)**.

---

### ✅ **3. Endpoint تسجيل الدخول وإرجاع التوكن**
دلوقتي لازم يكون عندنا `POST /auth/login` اللي بيأخذ بيانات تسجيل الدخول ويرجع التوكن:

```python
from fastapi import FastAPI, Depends, HTTPException, APIRouter
from fastapi.security import OAuth2PasswordRequestForm
from sqlalchemy.orm import Session
from database import get_db
from models import User
from passlib.context import CryptContext

bcrypt_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

router = APIRouter(prefix="/auth", tags=["auth"])

@router.post("/token")
async def login_for_access_token(form_data: OAuth2PasswordRequestForm = Depends(), db: Session = Depends(get_db)):
    user = db.query(User).filter(User.username == form_data.username).first()

    if not user or not bcrypt_context.verify(form_data.password, user.hashed_password):
        raise HTTPException(status_code=401, detail="Incorrect username or password")

    token = create_access_token(user.username, user.id, timedelta(minutes=30))
    return {"access_token": token, "token_type": "bearer"}
```
📌 **إيه اللي بيحصل هنا؟**
1. `OAuth2PasswordRequestForm` بيخلي FastAPI **تلقائيًا** يستقبل **username** و **password** من الفورم.
2. بندور على الـ user في الداتابيز ونتأكد إن الباسورد صح.
3. لو كل حاجة تمام، بنرجّع للمستخدم `access_token`.

---

### ✅ **4. استخدام `OAuth2PasswordBearer` للتحقق من الـ Token**
دلوقتي لازم يكون عندنا `Dependency` يقدر **يقرأ الـ token** المرسل مع كل طلب، ويفك تشفيره، ويتأكد إنه صالح:

```python
from fastapi.security import OAuth2PasswordBearer
from jose import jwt, JWTError

oauth2_bearer = OAuth2PasswordBearer(tokenUrl="auth/token")

async def get_current_user(token: str = Depends(oauth2_bearer)):
    try:
        payload = jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])
        username: str = payload.get("username")
        user_id: int = payload.get("user_id")

        if username is None or user_id is None:
            raise HTTPException(status_code=401, detail="Invalid token")

        return {"username": username, "user_id": user_id}

    except JWTError:
        raise HTTPException(status_code=401, detail="Invalid token")
```
📌 **إيه اللي بيحصل هنا؟**
1. **`oauth2_bearer` بياخد التوكن من الهيدر** تلقائيًا.
2. **`jwt.decode(token, SECRET_KEY, algorithms=[ALGORITHM])` بيفك تشفير التوكن**.
3. لو التوكن **غير صالح** أو **معدّل**، بنرمي **401 Unauthorized**.
4. لو التوكن **سليم**، بنرجّع بيانات المستخدم المستخرجة منه.

---

### ✅ **5. حماية Endpoints بالتوكن**
الآن بعد ما عندنا `get_current_user`، نقدر نحمي أي **endpoint** بسهولة عن طريق إضافة `Depends(get_current_user)`:

```python
@router.get("/protected-route")
async def protected_route(user: dict = Depends(get_current_user)):
    return {"message": f"Hello {user['username']}, welcome to the protected route!"}
```
📌 **إيه اللي بيحصل هنا؟**
- `Depends(get_current_user)` بيجبر المستخدم إنه **يبعت Access Token صالح** عشان يقدر يدخل.
- لو التوكن صالح، **المستخدم يقدر يدخل**.
- لو التوكن مش صالح، **هيطلع له `401 Unauthorized`**.

---

## 🎯 **ملخص سريع**
1. **`OAuth2PasswordBearer(tokenUrl="auth/token")`**
   - بيساعد FastAPI في استخراج التوكن من الهيدر **تلقائيًا**.

2. **إنشاء `JWT Token` عند تسجيل الدخول**
   - `create_access_token` بيعمل توكن يحتوي على بيانات المستخدم مع توقيت انتهاء الصلاحية.

3. **`login_for_access_token`**
   - بيأخذ الـ username & password، ويتحقق منهم، ويعيد التوكن.

4. **`get_current_user`**
   - بياخد التوكن المرسل، يفك تشفيره، ويتأكد إنه صالح.

5. **حماية الـ Endpoints**
   - أي **Endpoint حساس** لازم يعتمد على `Depends(get_current_user)` عشان يكون محمي.

---

## 💡 **ليه بنستخدم `OAuth2PasswordBearer`؟**
✅ بيخلّي FastAPI **تلقائيًا** تعرف منين تجيب التوكن من الهيدر.
✅ يسهّل **حماية الـ API** بدون ما نضطر نكتب كود إضافي لاستخراج التوكن يدويًا.
✅ **يتوافق مع كل Clients** اللي بتستخدم **Bearer Token Authentication** (زي Postman، Frontend Apps، وغيرها).


</div>